In [230]:
import pyspark
import pandas as pd
import sklearn
import findspark
import functools as ft
import pickle
import nltk
import string
import re
import tensorflow

In [231]:
from nltk.corpus import stopwords
from nltk.corpus import wordnet
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

from nltk.stem import WordNetLemmatizer

# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [232]:
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.model_selection import train_test_split

In [233]:
import pyspark
import findspark
from pyspark import SparkContext

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import Evaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [234]:
import os, sys
from pyspark.sql import SparkSession
import pyspark.sql.dataframe

findspark.init()

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

spark = SparkSession.builder.master('local[*]').appName("ml_example").getOrCreate()

sc = spark.sparkContext

In [235]:
from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import LinearSVCTrainingSummary

from pyspark.mllib.regression import LabeledPoint

from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.mllib.classification import NaiveBayes
from pyspark.mllib.classification import NaiveBayesModel

In [236]:


psy_df = pd.read_csv('data\Youtube01-Psy.csv')
katy_df = pd.read_csv('data\Youtube02-KatyPerry.csv')
lmfao_df = pd.read_csv('data\Youtube03-LMFAO.csv')
eminem_df = pd.read_csv('data\Youtube04-Eminem.csv')
shakira_df = pd.read_csv('data\Youtube05-Shakira.csv')

dataframes = [psy_df, katy_df, lmfao_df, eminem_df, shakira_df]

main_df = ft.reduce(lambda left, right: pd.concat([left, right], axis=0, ignore_index=True), dataframes)
main_df

,COMMENT_ID,AUTHOR,DATE,CONTENT,label
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1
...,...,...,...,...,...
1951,_2viQ_Qnc6-bMSjqyL1NKj57ROicCSJV5SwTrw-RFFA,Katie Mettam,2013-07-13T13:27:39.441000,I love this song because we sing it at Camp al...,0
1952,_2viQ_Qnc6-pY-1yR6K2FhmC5i48-WuNx5CumlHLDAI,Sabina Pearson-Smith,2013-07-13T13:14:30.021000,I love this song for two reasons: 1.it is abou...,0
1953,_2viQ_Qnc6_k_n_Bse9zVhJP8tJReZpo8uM2uZfnzDs,jeffrey jules,2013-07-13T12:09:31.188000,wow,0
1954,_2viQ_Qnc6_yBt8UGMWyg3vh0PulTqcqyQtdE7d4Fl0,Aishlin Maciel,2013-07-13T11:17:52.308000,Shakira u are so wiredo,0


## Steps to Follow:-
1. Remove Stopwords
2. Remove emojis
3. Remove punctuation
4. Lemmatization
5. Tokenization
6. Scaling
7. Implement Spark

In [237]:
main_df['label'].value_counts()

1    1005
0     951
Name: label, dtype: int64

In [238]:
main_df.drop(['COMMENT_ID', 'AUTHOR', 'DATE'], axis=1, inplace=True)
main_df

,CONTENT,label
0,"Huh, anyway check out this you[tube] channel: ...",1
1,Hey guys check out my new channel and our firs...,1
2,just for test I have to say murdev.com,1
3,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,watch?v=vtaRGgvGtWQ Check this out .﻿,1
...,...,...
1951,I love this song because we sing it at Camp al...,0
1952,I love this song for two reasons: 1.it is abou...,0
1953,wow,0
1954,Shakira u are so wiredo,0


## Adding single letters to the list of stopwords

In [239]:
alphabet_list = list(string.ascii_lowercase)
stop_words = stopwords.words('english')

for i in range(len(alphabet_list)):
    if (alphabet_list[i] in stop_words):
        continue
    else:
        stop_words.append(alphabet_list[i])
        
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## Removing Punctuations and Random Characters

In [240]:
def clean_text(df, text):
    df[text] = df[text].str.lower()
    df[text] = df[text].apply(lambda elem: re.sub(r'@[A-Za-z0-9]+', '', elem))              # Removing all @mentions
    df[text] = df[text].apply(lambda elem: re.sub(r'https?:\/\/\S+', '', elem))             # Removing all links
    df[text] = df[text].apply(lambda elem: re.sub(r'#', '', elem))                          # Removing the hashtag symbol
    df[text] = df[text].apply(lambda elem: re.sub(r"([^0-9A-Za-z \t])", "", elem))          # Removing all non-alphanumeric symbols
    df[text] = df[text].apply(lambda elem: re.sub(r"\w*\d\w*", "", elem))                   # Removing all numbers
    df[text] = df[text].apply(lambda elem: re.sub(r"^\d+\w|^\w+\d+\w|^\w+\d$", "", elem))
    
    return df

In [241]:
clean_df = clean_text(main_df, 'CONTENT')
clean_df

,CONTENT,label
0,huh anyway check out this youtube channel,1
1,hey guys check out my new channel and our firs...,1
2,just for test i have to say murdevcom,1
3,me shaking my sexy ass on my channel enjoy,1
4,watchvvtarggvgtwq check this out,1
...,...,...
1951,i love this song because we sing it at camp al...,0
1952,i love this song for two reasons is about afr...,0
1953,wow,0
1954,shakira u are so wiredo,0


## Removing Stopwords

In [242]:
def remove_stopwords(df, text):
    ABC = list((df[text]))
    clean_lst = []
    for i in range(ABC.__len__()):
        result_words = [word for word in ABC[i].split(' ') if word not in stop_words]
        str_result = ' '.join(result_words)
        clean_lst.append(str_result)
        
    cleaner_df = pd.DataFrame(clean_lst, columns=['CONTENT'])
    
    return cleaner_df

cleaner_df = remove_stopwords(clean_df, 'CONTENT')
cleaner_df.head(60)

cleaner_df = cleaner_df.dropna()

## Removing Emojis

In [243]:
cleaner_df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
# cleaner_df.to_csv('data/temp_data.csv',index=False)

,CONTENT
0,huh anyway check youtube channel
1,hey guys check new channel first vid us monke...
2,test say murdevcom
3,shaking sexy ass channel enjoy
4,watchvvtarggvgtwq check
...,...
1951,love song sing camp time
1952,love song two reasons africa born beautiful ...
1953,wow
1954,shakira wiredo


## Lemmatization

In [244]:
lemmatizer = WordNetLemmatizer()



def get_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {
        'V': wordnet.VERB,
        'N': wordnet.NOUN,
        'J': wordnet.ADJ,
        'R': wordnet.ADV
    }
    
    return tag_dict.get(tag)

def lem_df(df, text):
    lst_sent = list(df[text])
    lem_lst = []
    for i in range(len(lst_sent)):
        input = nltk.word_tokenize(lst_sent[i])
        result_words = [lemmatizer.lemmatize(word) for word in input]
        str_result = ' '.join(result_words)
        lem_lst.append(str_result)
        
    cleanest_df = pd.DataFrame(lem_lst, columns=['CONTENT'])
    return cleanest_df

lemmatized_df = lem_df(cleaner_df, 'CONTENT')
lemmatized_df = lemmatized_df.join(clean_df['label'])
lemmatized_df.to_csv('data/comment_data.csv', index=False)

## TFIDF Vectorization

In [245]:
docs = spark.read.csv('data/comment_data.csv', header=True, inferSchema=True)

tokenize = Tokenizer(inputCol='CONTENT', outputCol='WORDS')
words_data = tokenize.transform(docs.dropna())

hashingTF = HashingTF(inputCol="WORDS", outputCol="tf_features", numFeatures=20)
featurized_data = hashingTF.transform(words_data)

idf = IDF(inputCol="tf_features", outputCol="features")
idfModel = idf.fit(featurized_data)
rescaledData = idfModel.transform(featurized_data)

# rescaledData.show()
final_data = rescaledData.drop('CONTENT', 'WORDS', 'tf_features')
final_data.show()


+-----+--------------------+
|label|            features|
+-----+--------------------+
|    1|(20,[1,9,12,17],[...|
|    1|(20,[1,2,3,5,6,7,...|
|    1|(20,[2,6,8],[1.30...|
|    1|(20,[5,7,9,12,17]...|
|    1|(20,[1,8],[1.0256...|
|    1|(20,[1,2,3,5,8,11...|
|    1|(20,[2,12],[1.304...|
|    0|(20,[0,1,2,5,13,1...|
|    1|(20,[1,12,14,18],...|
|    1|(20,[1,10,12,13,1...|
|    1|(20,[2,8],[1.3049...|
|    1|(20,[2,3,4,5,6,7,...|
|    1|(20,[2,10],[2.609...|
|    1|(20,[8,10],[1.166...|
|    1|(20,[0,1,2,3,4,6,...|
|    0|(20,[3,4,13],[1.2...|
|    1|(20,[0,1,2,3,4,5,...|
|    0|(20,[1,3,4,11,13,...|
|    1|(20,[2,3,12],[1.3...|
|    1|(20,[5,13,19],[1....|
+-----+--------------------+
only showing top 20 rows



### Splitting training and testing data

In [252]:
splits = final_data.randomSplit([0.75, 0.25], 69)
train_data = splits[0]
test_data = splits[1]

train_rdd = train_data.rdd
train_rdd_final = train_rdd.map(lambda x: LabeledPoint(x[0], x[1]))
print(train_rdd_final.collect())
# print(train_rdd.collect())
labels = train_rdd.map(
    lambda doc: doc['label']
)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 89.0 failed 1 times, most recent failure: Lost task 0.0 in stage 89.0 (TID 90) (DHRUVARSHI-DELL-G3-3579 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Users\dhruv\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 686, in main
  File "C:\Users\dhruv\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 678, in process
  File "C:\Users\dhruv\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pyspark\python\lib\pyspark.

In [251]:
model = NaiveBayes.train(train_rdd)
labels_and_preds = labels.zip(model.predict(rescaledData)).map(
    lambda x: {"actual": x[0], "predicted": float(x[1])})

ValueError: `data` should be an RDD of LabeledPoint

In [ ]:
predictionAndLabel = test_data.map(lambda p: (model.predict(p.features), p.label))
accuracy = 1.0 * predictionAndLabel.filter(lambda pl: pl[0] == pl[1]).count() / test_data.count()
print('model accuracy {}'.format(accuracy))

In [ ]:
""" evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy") 
nbaccuracy = evaluator.evaluate(predictions_df) 
print("Test accuracy = " + str(nbaccuracy)) """

Prediction Accuracy:  0.6069469835466179
